In [7]:
import os, re, subprocess, requests, json, random
from pymediainfo import MediaInfo
from datetime import datetime
import subprocess
from smms import *
from utils import *
import base64, zlib
crf=20.0
qcomp=0.6
aq_mode=3
aq_s=0.6
newline="\r\n"

PROCUCT_DEST_DIR=r'/Users/zl/Movies'

#https://api.rhilip.info/ptgen.html
DOUBAN_ID="1303275"

SRC="Possessed 1947 1080p Blu-ray AVC DTS-HD MA 2.0-DiY@PTHome".strip().replace(" ",".")
print("SRC: ",SRC)
REMUX_DIRNAME="Possessed.1947.1080p.Bluray.AVC.Remux.DTS-HD.MA.2.0-PTH"
# if "AVC.Remux" in REMUX_DIRNAME and "DTS-HD.MA" in REMUX_DIRNAME:
#     ENCODE_FILENAME=REMUX_DIRNAME.replace("AVC.Remux","REMUX").replace("DTS-HD.MA.5.1","DTS").replace("DTS-HD.MA.7.1","DTS")+".mkv"
# else:
ENCODE_FILENAME="Possessed.1947.1080p.Bluray.REMUX.DTS-PTH.mkv"

print(REMUX_DIRNAME,"==>",ENCODE_FILENAME)
MOVIE=os.path.join(PROCUCT_DEST_DIR, REMUX_DIRNAME, REMUX_DIRNAME+".mkv")

ACKNOWLEDGEMENT='''视频原盘来自@CiNEMATiC，Warner Archive Collection版
字幕取自subhd@YIFY，自行ORC中文字幕，逐条修订翻译，时间轴，并合成双语字幕。
'''

SOUNDTRACK="【英语】【简|繁|简英|繁英|英字幕】"
# SOUNDTRACK="【英语】【简|繁|简英|繁英|英字幕】"

SUBTITLE_GZ=1 #官字组

LOG_DIR=os.getcwd()

SRC:  Possessed.1947.1080p.Blu-ray.AVC.DTS-HD.MA.2.0-DiY@PTHome
Possessed.1947.1080p.Bluray.AVC.Remux.DTS-HD.MA.2.0-PTH ==> Possessed.1947.1080p.Bluray.REMUX.DTS-PTH.mkv


In [8]:
cropx=240
cropy=0

x264_filename=ENCODE_FILENAME.replace("REMUX", "x264")
x264_dirname=x264_filename.replace(".mkv","")
x264_fullpath=os.path.join(PROCUCT_DEST_DIR, x264_dirname, x264_filename)
# x265_filename="".join(re.findall(r'([^/\\]*$)', MOVIE)).replace("REMUX", "x265.10bit")
x265_filename=ENCODE_FILENAME.replace("REMUX", "x265.10bit")
x265_dirname=x265_filename.replace(".mkv","")
x265_fullpath=os.path.join(PROCUCT_DEST_DIR, x265_dirname, x265_filename)

remux_filename=REMUX_DIRNAME+".mkv"
remux_fullpath=os.path.join(PROCUCT_DEST_DIR, REMUX_DIRNAME, REMUX_DIRNAME+".mkv")

cmdcommon_before=f'''source pthvs.sh
export CRF={crf}
export QC={qcomp}
export AQS={aq_s}
export REMUX_FILE={remux_fullpath}
export CROPX={cropx}
export CROPY={cropy}
export ISREMUX=False'''

cmdcommon_after=""
cmd264=""
cmd265=""

if "DTS" in REMUX_DIRNAME:
    dtscore="-c:a dca -core_only true"
    acodec="-bsf:a dca_core -c:a copy"
else:
    if "LPCM" in REMUX_DIRNAME:
        acodec="-c:a flac"
    else:
        acodec="-bsf:a dca_core -c:a dts"

cmd264+=f'''
export PRODUCT_FILE={x264_fullpath}
mkdir -p $(dirname $PRODUCT_FILE)
ffmpeg -n {dtscore} -i $REMUX_FILE -c:v libx264 -profile:v high -filter:v "crop=in_w-2*$CROPX:in_h-2*$CROPY" -preset slower \
-x264-params ref=4:crf=$CRF:qcomp=$QC:aq-mode=3:aq-strength=$AQS:bframes=11:me=umh:subme=11:merange=48:no-fast-pskip=1:\
no-dct-decimate=1:direct=auto:psy-rd=1.00,0.00:vbv-bufsize=78125:vbv-maxrate=62500:deblock=-3,-3:b-adapt=2:keyint=240:\
min-keyint=1:no-mbtree=1:trellis=2:chroma-qp-offset=-1:rc-lookahead=72 -max_muxing_queue_size 1024 {acodec} \
-c:s copy -max_muxing_queue_size 1024 -map 0 -map -v -map V -report $PRODUCT_FILE 2>&1 | tee $(dirname $PRODUCT_FILE).log
'''
#pthpipe -i $REMUX_FILE -cx $CROPX -cy $CROPY|pth264 -crf $CRF -qc $QC -aqs $AQS -o $STREAM_FILE |& tee 

cmd265+=f'''
export PRODUCT_FILE={x265_fullpath}
mkdir -p $(dirname $PRODUCT_FILE)
ffmpeg -n {dtscore} -i $REMUX_FILE -pix_fmt yuv420p10le -c:v libx265 -profile:v main10 -filter:v "crop=in_w-2*$CROPX:in_h-2*$CROPY" \
-preset slower -x265-params crf=$CRF:qcomp=$QC:aq-mode=3:aq-strength=$AQS:pools=36:numa=48:rd=4:psy-rd=2.0:psy-rdoq=1.0:\
rdoq-level=2:deblock=-1,-1:cbqpoffs=-1:crqpoffs=-3:ctu=32:qg-size=8:no-sao=1:no-sao-non-deblock=1:selective-sao=0:\
tu-intra-depth=4:tu-inter-depth=4:no-opt-qp-pps=1:no-opt-ref-list-length-pps=1:aud=1:repeat-headers=1:me=3:subme=5:\
merange=48:b-intra=1:limit-tu=0:no-rect=1:no-amp=1:no-open-gop=1:pools=+:keyint=240:min-keyint=1:bframes=8:max-merge=4:\
ref=4:weightb=1:rc-lookahead=72:scenecut=40:no-strong-intra-smoothing=1:vbv-bufsize=160000:vbv-maxrate=160000:input-depth=10 \
-max_muxing_queue_size 1024 {acodec} -c:s copy -max_muxing_queue_size 1024 -map 0 -map -v -map V -report $PRODUCT_FILE 2>&1 | tee $(dirname $PRODUCT_FILE).log
'''

cmdremux=f'export PRODUCT_FILE={remux_fullpath}\nexport ISREMUX=True\n'
    
transcodecmd="mkvpropedit $PRODUCT_FILE --add-track-statistics-tags\n"
cmdcommon_after+=get_captures("$(dirname $PRODUCT_FILE).captures","$PRODUCT_FILE")+"\n"
cmdcommon_after+=f"transmission-create -p -o $(dirname $PRODUCT_FILE).torrent -t https://www.pthome.net/announce.php $(dirname $PRODUCT_FILE)"   

cmdcommon_after+=f'''
sed -i "s/\[lib\(x26[[:digit:]]\).*\]/\1 [info]:/g" $(dirname $PRODUCT_FILE).log

python3 <<!
import os, re, subprocess, requests, json, random
from pymediainfo import MediaInfo
from datetime import datetime
from smms import *
from utils import *
import base64, zlib

DOUBAN_ID="{DOUBAN_ID}"
ACKNOWLEDGEMENT="""{ACKNOWLEDGEMENT}"""
SOUNDTRACK="{SOUNDTRACK}"
SRC="{SRC}"
SUBTITLE_GZ={SUBTITLE_GZ}

metainfo=getmetainfo(DOUBAN_ID)
metainfo["douban_link"]=f"https://movie.douban.com/subject/{DOUBAN_ID}/"
smms = SMMS('hzlclock', 'Hezl1996')
smms.get_api_token()
picbed=upload_folder("$(dirname $PRODUCT_FILE).captures",smms)

with open(f"$(dirname $PRODUCT_FILE).bbcode.base64.txt","w") as f:
    f.write(getbase64json("$PRODUCT_FILE", src=SRC, logname="$(dirname $PRODUCT_FILE).log",\
            picbed=picbed, metainfo=metainfo, is_remux=$ISREMUX,\
            tracks=SOUNDTRACK,gz=SUBTITLE_GZ,ack=ACKNOWLEDGEMENT))
!
'''

# with open(x264_filename+".sh","w") as f:
#     f.write(f"{cmdcommon_before}\n{cmd264}\n{transcodecmd}\n{cmdcommon_after}")
# with open(x265_filename+".sh","w") as f:
#     f.write(f"{cmdcommon_before}\n{cmd265}\n{transcodecmd}\n{cmdcommon_after}")
# with open(remux_filename+".sh","w") as f:
#     f.write(f"{cmdcommon_before}\n{cmdremux}\n{cmdcommon_after}")
    
with open(remux_filename+".sh","w") as f:
    f.write(f"{cmdcommon_before}\n{cmdremux}\n{cmdcommon_after}")
    f.write(f"\n{cmd264}\n{transcodecmd}\n{cmdcommon_after}")
    f.write(f"\n{cmd265}\n{transcodecmd}\n{cmdcommon_after}")
    
print(f'bash {x264_filename}.sh;\nbash {x265_filename}.sh;\nbash {remux_filename}.sh;')

#ffmpeg -i dest.mkv -i remux.mkv -map 0:v -map 0:s -map 1:a -c:v copy -c:a dts -strict -2 dtsout.mkv


bash Possessed.1947.1080p.Bluray.x264.DTS-PTH.mkv.sh;
bash Possessed.1947.1080p.Bluray.x265.10bit.DTS-PTH.mkv.sh;
bash Possessed.1947.1080p.Bluray.AVC.Remux.DTS-HD.MA.2.0-PTH.mkv.sh;
